# Installing and Loading Libraries

In [1]:
!pip install neo4j

# Creating Function

In [2]:
import pandas as  pd

In [3]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [30]:
def result_to_dataframe(result):
    
    if result == []:
        return 'No records Found'
    
    else:
    
        data = { column:[] for column, _ in result[0].items()}

        for record in result:

            for col, val in record.items():

                data[col].append(val)

        return pd.DataFrame(data)

In [5]:
db =  Neo4jConnection("bolt://localhost:7687", "neo4j", "password")

# Query 1 Person role in that Movie

In [6]:
q  = '''match (p:Person) -[r]-> (m:Movie)
        return p.name as person ,type(r) as role ,m.title as movie'''

result = db.query(q)

result_to_dataframe(result)

,person,role,movie
0,Emil Eifrem,ACTED_IN,The Matrix
1,Joel Silver,PRODUCED,The Matrix
2,Lana Wachowski,DIRECTED,The Matrix
3,Andy Wachowski,DIRECTED,The Matrix
4,Hugo Weaving,ACTED_IN,The Matrix
...,...,...,...
242,Bill Paxton,ACTED_IN,A League of Their Own
243,Tom Hanks,ACTED_IN,A League of Their Own
244,Rosie O'Donnell,ACTED_IN,A League of Their Own
245,Geena Davis,ACTED_IN,A League of Their Own


# Query 2 : List of Movie 

In [37]:
q  = '''match (m:Movie)
        return m.title as movie, m.tagline as tagline,m.released as released
        order by released desc'''

result = db.query(q)
result_to_dataframe(result)

,movie,tagline,released
0,Cloud Atlas,Everything is connected,2012
1,Ninja Assassin,Prepare to enter a secret world of assassins,2009
2,Speed Racer,Speed has no limits,2008
3,Frost/Nixon,400 million people were waiting for the truth.,2008
4,Charlie Wilson's War,A stiff drink. A little mascara. A lot of nerv...,2007
5,RescueDawn,Based on the extraordinary true story of one m...,2006
6,The Da Vinci Code,Break The Codes,2006
7,V for Vendetta,Freedom! Forever!,2006
8,The Polar Express,This Holiday Season… Believe,2004
9,The Matrix Reloaded,Free your mind,2003


# Query 3 : List of Person  along with roles

In [8]:
q  = '''match (p:Person) -[r] -> (m:Movie)
        return p.name as person , p.born as birth_year, type(r) as roles'''

result = db.query(q)
result_to_dataframe(result)

,person,birth_year,roles
0,Emil Eifrem,1978.0,ACTED_IN
1,Joel Silver,1952.0,PRODUCED
2,Lana Wachowski,1965.0,DIRECTED
3,Andy Wachowski,1967.0,DIRECTED
4,Hugo Weaving,1960.0,ACTED_IN
...,...,...,...
242,Bill Paxton,1955.0,ACTED_IN
243,Tom Hanks,1956.0,ACTED_IN
244,Rosie O'Donnell,1962.0,ACTED_IN
245,Geena Davis,1956.0,ACTED_IN


# Query 4 : Top 5 People who worked most Number of movies as Director

In [9]:
q  = '''match (p) -[:DIRECTED]- (m)
        with p ,count(m) as no_of_movies
        return p.name as Person, no_of_movies
        order by no_of_movies desc
        Limit 5'''

result = db.query(q)
result_to_dataframe(result)

,Person,no_of_movies
0,Andy Wachowski,5
1,Lana Wachowski,5
2,None,3
3,Rob Reiner,3
4,Ron Howard,3


# Query 5: List of Person who worked as Director and Producerd in Same Movie
# ![Picture](Pictures/ql.png)

In [11]:
q  = '''match(p:Person) -[:DIRECTED]-> (m:Movie)
        match(p) -[:PRODUCED] -> (m)
        return p.name as person, p.born as birth_year, m.title as movie, m.released'''

result = db.query(q)
result_to_dataframe(result)

,person,birth_year,movie,m.released
0,Cameron Crowe,1957,Jerry Maguire,2000
1,Rob Reiner,1947,When Harry Met Sally,1998
2,Nancy Meyers,1949,Something's Gotta Give,2003


# Query 6: Person of worked more than 6 movies . Give names of person and movies he/ she worked

![Picture](Pictures/q6.png)

In [16]:
q =  """match (p:Person) --> (m:Movie)
        with p ,collect(m) as movies
        where size(movies) > 6
        UNWIND movies as m
        return p.name as Person ,m.title as Movie"""

result = db.query(q)
result_to_dataframe(result)

,Person,Movie
0,Lana Wachowski,The Matrix
1,Lana Wachowski,The Matrix Reloaded
2,Lana Wachowski,The Matrix Revolutions
3,Lana Wachowski,Cloud Atlas
4,Lana Wachowski,V for Vendetta
5,Lana Wachowski,V for Vendetta
6,Lana Wachowski,Speed Racer
7,Lana Wachowski,Speed Racer
8,Lana Wachowski,Ninja Assassin
9,Andy Wachowski,The Matrix


## Query 7 : Person who worked as writer and actor in same movie

![](Pictures/q7.png)

In [13]:
q =  """match (p:Person) -[:ACTED_IN]-> (m)
        match (p) -[:WROTE] -> (m)
        return p.name as Person ,m.title as Movie"""

result = db.query(q)
result_to_dataframe(result)

,Person,Movie
0,Aaron Sorkin,A Few Good Men


## Query 8 : Person who worked as director and actor in same movie

![](Pictures/q8.png)

In [14]:
q =  """match (p:Person) -[:ACTED_IN]-> (m)
        match (p) -[:DIRECTED] -> (m)
        return p.name as Person ,m.title as Movie"""

result = db.query(q)
result_to_dataframe(result)

,Person,Movie
0,Tom Hanks,That Thing You Do
1,Clint Eastwood,Unforgiven
2,Danny DeVito,Hoffa


# Query 9 Getting data to any Person Provide by User along work role in that movie:

For EG:
1. Lana Wachowski
2. Tom Hanks
3. Keanu Reeves

In [34]:
person =  input('Enter the name of Person: ')

q = f"""match (p:Person {{name:'{person}'}}) -[r]-> (m)
    return m.title as Movie , type(r) as Role"""

result = db.query(q)
result_to_dataframe(result)


Enter the name of Person: Tom Hanks


,Movie,Role
0,You've Got Mail,ACTED_IN
1,Apollo 13,ACTED_IN
2,Joe Versus the Volcano,ACTED_IN
3,That Thing You Do,ACTED_IN
4,Cloud Atlas,ACTED_IN
5,The Da Vinci Code,ACTED_IN
6,Sleepless in Seattle,ACTED_IN
7,A League of Their Own,ACTED_IN
8,The Green Mile,ACTED_IN
9,Charlie Wilson's War,ACTED_IN


# Query 10 Getting data Person , Role and Movie which is/contains movie provide by User 

For EG:
1. The Matrix
2. The Matrix Reloaded
3. The Matrix Revolutions
4. Speed Racer
5. Cloud Atlas

In [39]:
movie = input('Enter the movie title which has or contains: ')

q = f"""match (p) -[r]-> (m:Movie)
    where m.title contains '{movie}'
    return p.name as Person,type(r) as Role ,m.title as Movie"""

result = db.query(q)
result_to_dataframe(result)

Enter the movie title which has or contains: Cloud Atlas


,Person,Role,Movie
0,Tom Hanks,ACTED_IN,Cloud Atlas
1,Jim Broadbent,ACTED_IN,Cloud Atlas
2,Tom Tykwer,DIRECTED,Cloud Atlas
3,Lana Wachowski,DIRECTED,Cloud Atlas
4,Jessica Thompson,REVIEWED,Cloud Atlas
5,Halle Berry,ACTED_IN,Cloud Atlas
6,Hugo Weaving,ACTED_IN,Cloud Atlas
7,Andy Wachowski,DIRECTED,Cloud Atlas


# Query 11 Getting data Person , character, workage and Movie namewho is actor  which is/contains movie provide by User

For EG:
1. The Matrix
2. The Matrix Reloaded
3. The Matrix Revolutions
4. Speed Racer
5. Cloud Atlas

In [41]:
movie = input('Enter the movie title which has or contains: ')

q = f"""match (p) -[r:ACTED_IN]-> (m:Movie)
    where m.title contains '{movie}'
    return p.name as Person ,r.roles as Character, m.released - p.born as WorkAge, m.title as Movie"""

result = db.query(q)
result_to_dataframe(result)

Enter the movie title which has or contains: Matrix


,Person,Character,WorkAge,Movie
0,Emil Eifrem,[Emil],21,The Matrix
1,Hugo Weaving,[Agent Smith],39,The Matrix
2,Laurence Fishburne,[Morpheus],38,The Matrix
3,Carrie-Anne Moss,[Trinity],32,The Matrix
4,Keanu Reeves,[Neo],35,The Matrix
5,Hugo Weaving,[Agent Smith],43,The Matrix Reloaded
6,Laurence Fishburne,[Morpheus],42,The Matrix Reloaded
7,Carrie-Anne Moss,[Trinity],36,The Matrix Reloaded
8,Keanu Reeves,[Neo],39,The Matrix Reloaded
9,Hugo Weaving,[Agent Smith],43,The Matrix Revolutions


In [ ]:
def graphs_to_csv(result):
    
    dct = {}
    
    for var in result:
        dct = { k[0]:{} for k in var.items() }
        break
        
    
    for i in result:
        d = i.items()
        for var , data in d:
            label = ' '.join(list(data.labels))
            dct[var][label] = {}
            properties = data._properties.keys()
            
            if not label in dct:
                dct[var][label] = { k : [] for k in properties }

            else:
                for i in properties:
                    if not i in dct[label]:
                        dct[var][label][i] = []
            
    for i in result:
        
        d = i.items()
        for var , data in d:
            label = ' '.join(list(data.labels))
            properties = data._properties
            
            for k in dct[var][label] :

                v = properties.get(k,float('nan'))

                dct[var][label][k].append(v)
    
    for var in dct:
        for label , v in dct[var].items():
            dct[var][label] = pd.DataFrame(v)
        
    return dct